# Simulated Supply Chain DataFrame

Project Introduction

In an increasingly complex and interconnected world, efficient logistics and supply chain management have become paramount to the success of businesses and economies. From ensuring timely deliveries to optimizing operational costs, the challenges within logistics are vast and varied. This project undertakes a rigorous analysis of simulated logistics data to unravel key performance indicators, uncover inefficiencies, and propose actionable insights to enhance overall supply chain operations.

Our simulated dataset mirrors real-world logistics challenges, with data points that include distance traveled, fuel costs, delivery times, driver consistency, customer satisfaction, traffic conditions, and more. By leveraging machine learning techniques such as clustering, dimensionality reduction, and predictive modeling, this project seeks to provide a comprehensive framework for identifying areas of improvement within the logistics landscape.

Our objective is twofold: first, to develop a deeper understanding of the factors impacting operational costs and delivery efficiency, and second, to construct predictive models capable of guiding decision-making to optimize these factors. This multi-faceted project is divided into several stages, each building upon the previous one to achieve a holistic view of the logistics operation.

In [36]:
import numpy as np
import pandas as pd
import string
import random
from faker import Faker
import sys
from geopy.distance import geodesic

In [37]:
import os

In [38]:

# Assuming 'scripts' is in the same parent directory as 'notebooks'
sys.path.append(os.path.abspath("../scripts"))

from data_cleaning import load_data, clean_data

In [39]:
# Initialize Faker and define number of records
fake = Faker()
num_records = 10000

# Define lists for categorical variables
weather_conditions = ["Clear", "Light Rain", "Heavy Rain", "Snow", "Fog"]
traffic_conditions = ["Light", "Moderate", "Heavy", "Severe"]
experience_levels = ["Junior", "Intermediate", "Senior"]
delivery_windows = ["Morning", "Afternoon", "Evening", "Overnight"]
package_types = ["Standard", "Fragile", "Perishable", "Oversized"]
route_types = ["Interstate", "Urban", "Suburban"]
truck_types = ["Box Truck", "Semi", "Flatbed"]
satisfaction_levels = ["Very Satisfied", "Satisfied", "Neutral", "Dissatisfied", "Very Dissatisfied"]
fuel_types = ["Diesel", "Gasoline"]

# Define major U.S. airports with coordinates for distance calculations
airports = ["JFK", "BOS", "MIA", "DCA", "ATL", "CLT", "ORD", "DTW", "MSP", "STL", "CMH", "CLE", "DFW", "IAH", "MCO", 
            "BNA", "LAX", "SFO", "SEA", "LAS", "PHX", "SAN", "DEN", "SLC", "BOI"]

airport_coords = {
    "JFK": (40.6413, -73.7781), "BOS": (42.3656, -71.0096), "MIA": (25.7959, -80.2870), "DCA": (38.8512, -77.0402),
    "ATL": (33.6407, -84.4277), "CLT": (35.2140, -80.9431), "ORD": (41.9742, -87.9073), "DTW": (42.2162, -83.3554),
    "MSP": (44.8827, -93.2218), "STL": (38.7490, -90.3748), "CMH": (39.9980, -82.8919), "CLE": (41.4121, -81.8498),
    "DFW": (32.8998, -97.0403), "IAH": (29.9902, -95.3368), "MCO": (28.4312, -81.3081), "BNA": (36.1263, -86.6774),
    "LAX": (33.9416, -118.4085), "SFO": (37.6213, -122.3790), "SEA": (47.4502, -122.3088), "LAS": (36.0840, -115.1537),
    "PHX": (33.4342, -112.0119), "SAN": (32.7338, -117.1933), "DEN": (39.8561, -104.6737), "SLC": (40.7899, -111.9791),
    "BOI": (43.5644, -116.2227)
}

# Generate random data with realistic constraints
data = {
    "Route ID": [f"{random.choice(airports)}/{random.choice(airports)}" for _ in range(num_records)],
    "Driver ID": [f"{random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZ')}{random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZ')}{np.random.randint(1000, 9999)}" for _ in range(num_records)],
    "Manifest": [str(np.random.randint(1000000, 9999999)) for _ in range(num_records)],
    "Delivery Duration (hours)": np.random.uniform(1, 10, num_records),
    "Date": np.random.choice(pd.date_range(start="2023-01-01", end="2023-12-31", freq='D'), num_records),
    "Fuel Price per Gallon (USD)": np.random.choice([3.5, 4.0, 4.5], num_records),
    "Actual Distance (miles)": np.random.uniform(100, 3000, num_records),
    "Planned Distance (miles)": np.random.uniform(100, 3000, num_records),
    "Weather Conditions": np.random.choice(weather_conditions, num_records),
    "Traffic Conditions": np.random.choice(traffic_conditions, num_records),
    "Driver Ratings": np.random.uniform(1, 5, num_records),
    "Customer Satisfaction": np.random.choice(satisfaction_levels, num_records),
    "Delays (hours)": np.random.uniform(0, 5, num_records),
    "Warehouse Storage Costs (USD)": np.random.uniform(50, 200, num_records),
    "Truck Maintenance Costs (USD)": np.random.uniform(500, 2000, num_records),
    "Load Type": np.random.choice(package_types, num_records),
    "Load Weight (tons)": np.random.uniform(0.5, 20, num_records),
    "Route Type": np.random.choice(route_types, num_records),
    "Truck Type": np.random.choice(truck_types, num_records),
    "Driver Experience": np.random.choice(experience_levels, num_records),
    "Delivery Window": np.random.choice(delivery_windows, num_records),
    "Truck Condition": np.random.randint(1, 6, num_records),
    "Labor Costs (USD)": np.random.uniform(20, 200, num_records),
    "Fuel Type": np.random.choice(fuel_types, num_records),
    "Toll Costs (USD)": np.random.uniform(0, 50, num_records),
    "Parking Costs (USD)": np.random.uniform(0, 30, num_records),
    "Time Idling (hours)": np.random.uniform(0, 2, num_records)  # Idling time in hours
}

# Calculated columns
data["Miles per Gallon"] = np.random.uniform(6, 8, num_records)
data["Calculated Fuel Cost (USD)"] = (data["Actual Distance (miles)"] / data["Miles per Gallon"]) * data["Fuel Price per Gallon (USD)"]
data["Idle Cost (USD)"] = data["Time Idling (hours)"] * data["Fuel Price per Gallon (USD)"] * 0.5
# Add Fuel Cost per Mile
data["Fuel Cost Per Mile"] = data["Calculated Fuel Cost (USD)"] / data["Actual Distance (miles)"]

# Total operational cost
data["Total Operational Cost (USD)"] = (
    data["Calculated Fuel Cost (USD)"] +
    data["Toll Costs (USD)"] +
    data["Warehouse Storage Costs (USD)"] +
    data["Truck Maintenance Costs (USD)"] +
    data["Parking Costs (USD)"] +
    data["Idle Cost (USD)"]
)

# DataFrame creation
logistics_df = pd.DataFrame(data)

# Rounding specific columns for readability
columns_to_round = [
    "Actual Distance (miles)", "Warehouse Storage Costs (USD)", "Truck Maintenance Costs (USD)", "Delivery Duration (hours)",
    "Toll Costs (USD)", "Parking Costs (USD)", "Idle Cost (USD)", "Total Operational Cost (USD)",
    "Calculated Fuel Cost (USD)", "Planned Distance (miles)", "Driver Ratings", "Delays (hours)", "Load Weight (tons)", "Fuel Cost Per Mile", "Labor Costs (USD)", "Time Idling (hours)", "Miles per Gallon"
]
logistics_df[columns_to_round] = logistics_df[columns_to_round].round(2)

# Calculating "Distance Difference" and ensuring it's non-negative
logistics_df["Distance Difference (miles)"] = (logistics_df["Actual Distance (miles)"] - logistics_df["Planned Distance (miles)"]).clip(lower=0)

# Renaming columns
logistics_df.rename(columns={
    "Delivery Duration (hours)": "Delivery Duration (hours)",
    "Fuel Price per Gallon (USD)": "Fuel Expenditure per Gallon (USD)",
    "Actual Distance (miles)": "Actual Distance (miles)",
    "Planned Distance (miles)": "Planned Distance (miles)",
    "Time Idling (hours)": "Idle Duration (hours)"
}, inplace=True)

# Create a DataFrame
logistics_df = pd.DataFrame(data)

# Filter columns to include only those present in logistics_df
existing_columns_to_round = [col for col in columns_to_round if col in logistics_df.columns]

# Apply rounding to existing columns only
logistics_df[existing_columns_to_round] = logistics_df[existing_columns_to_round].round(2)
# Rounding to 4 decimal places for specific columns

# Apply rounding
logistics_df[columns_to_round] = logistics_df[columns_to_round].round(2)

# Ensuring columns like "Weather Severity" aren't empty by assigning defaults
logistics_df["Weather Severity"] = np.random.choice([0, 1, 2, 3], size=10000)
logistics_df["Weather Severity Score"] = np.where(data["Weather Conditions"] == "Clear", 0, logistics_df["Weather Severity"])

# Stripping whitespace from text fields, if generated with extra spaces
logistics_df = logistics_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [40]:
print(logistics_df.head())

  Route ID Driver ID Manifest  Delivery Duration (hours)       Date  \
0  MIA/BNA    BC7706  3815805                       6.25 2023-02-28   
1  PHX/MCO    ZN7236  7299049                       5.67 2023-10-29   
2  MCO/MIA    AW5182  9497986                       1.31 2023-09-14   
3  MIA/DCA    XY7077  7232797                       2.38 2023-12-31   
4  LAS/SAN    HJ7731  2201569                       1.24 2023-05-02   

   Fuel Price per Gallon (USD)  Actual Distance (miles)  \
0                          4.0                  1690.79   
1                          3.5                  2693.86   
2                          4.0                   925.07   
3                          4.5                   660.51   
4                          4.5                  1135.09   

   Planned Distance (miles) Weather Conditions Traffic Conditions  ...  \
0                    862.13               Snow           Moderate  ...   
1                   1970.77         Heavy Rain           Moderate  ...

In [41]:
# Define paths to save/load data in the `data` directory
raw_data_path = '../data/raw/logistics_df.csv'
raw_excel_path = '../data/raw/logistics_df.xlsx'
cleaned_data_path = '../data/processed/cleaned_logistics_data.csv'
engineered_data_path = '../data/processed/engineered_data.csv'

# Ensure the `raw` directory exists within `data`
raw_directory = os.path.dirname(raw_data_path)
if not os.path.exists(raw_directory):
    os.makedirs(raw_directory)
    print(f"Directory created at {raw_directory}")

# Example: Saving a DataFrame to the `data/raw` directory# Sample data
logistics_df.to_csv(raw_data_path, index=False)
print(f"Data saved successfully to {raw_data_path}")

# Save the DataFrame to an Excel file
logistics_df.to_excel(raw_excel_path, index=False)  # index=False to avoid saving the index as a column

Data saved successfully to ../data/raw/logistics_df.csv


In [42]:
# Load the dataset
logistics_df = pd.read_csv('../data/raw/logistics_df.csv')

# Display first few rows
logistics_df.head()

,Route ID,Driver ID,Manifest,Delivery Duration (hours),Date,Fuel Price per Gallon (USD),Actual Distance (miles),Planned Distance (miles),Weather Conditions,Traffic Conditions,...,Toll Costs (USD),Parking Costs (USD),Time Idling (hours),Miles per Gallon,Calculated Fuel Cost (USD),Idle Cost (USD),Fuel Cost Per Mile,Total Operational Cost (USD),Weather Severity,Weather Severity Score
0,MIA/BNA,BC7706,3815805,6.25,2023-02-28,4.0,1690.79,862.13,Snow,Moderate,...,14.42,21.00,0.46,7.03,962.65,0.93,0.57,1659.22,2,2
1,PHX/MCO,ZN7236,7299049,5.67,2023-10-29,3.5,2693.86,1970.77,Heavy Rain,Moderate,...,39.82,7.16,1.65,7.48,1259.91,2.89,0.47,2237.47,2,2
2,MCO/MIA,AW5182,9497986,1.31,2023-09-14,4.0,925.07,1337.27,Heavy Rain,Heavy,...,24.93,10.30,0.83,7.76,476.54,1.66,0.52,1883.15,2,2
3,MIA/DCA,XY7077,7232797,2.38,2023-12-31,4.5,660.51,1833.61,Heavy Rain,Moderate,...,22.28,8.18,0.68,6.83,435.50,1.53,0.66,2538.15,1,1
4,LAS/SAN,HJ7731,2201569,1.24,2023-05-02,4.5,1135.09,428.03,Snow,Light,...,37.74,12.98,0.67,7.84,651.60,1.50,0.57,2169.99,1,1
